In [1]:
!pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
df1=pd.read_parquet('0000.parquet', engine='fastparquet')
df2=pd.read_parquet('0001.parquet', engine='fastparquet')
df3=pd.read_parquet('0002.parquet', engine='fastparquet')
df4=pd.read_parquet('0003.parquet', engine='fastparquet')
df5=pd.read_parquet('0004.parquet', engine='fastparquet')
df6=pd.read_parquet('0005.parquet', engine='fastparquet')
df7=pd.read_parquet('0006.parquet', engine='fastparquet')
df8=pd.read_parquet('0007.parquet', engine='fastparquet')
df9=pd.read_parquet('0008.parquet', engine='fastparquet')
df10=pd.read_parquet('0009.parquet', engine='fastparquet')

In [3]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:
df = pd.concat ([df1, df2, df3,df4,df5,df6,df7,df8,df9,df10])
feature_to_drop=['title','blocks.data','username','author_id','timestamp', 'id','blocks.type','comments.id','comments.timestamp','comments.parent_id','comments.text_markdown','comments.text_html','comments.images','comments.rating','comments.pluses',	'comments.minuses','comments.author_id','comments.username','rating','pluses','minuses','url']
df.drop(feature_to_drop, axis=1, inplace=True)
df = df.explode('tags')
import catboost as cb

In [5]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
df = df.sample(frac=0.007) # Get 30% of the data
len(df)  

7953

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
print(1)
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df[['text_markdown']], df['tags'], test_size=0.4, random_state=42)
print(1)
# Создание

1
1


In [6]:
model = CatBoostClassifier(
    depth=6,
    iterations=1000,
    
    learning_rate=0.01,
    loss_function='MultiClass',
    # Главная фишка катбуста - работа с категориальными признаками
    
    # ignored_features = ignored_features,
    
    # Регуляризация и ускорение
    colsample_bylevel=0.098,
    subsample=0.95,
    l2_leaf_reg=1,
    max_bin=187,
    
    # Параметры скорения  
    thread_count=-1,
    bootstrap_type="Bernoulli", 
    
    # Важное!
    auto_class_weights="SqrtBalanced",
    early_stopping_rounds=10,
    verbose=10,
    text_features=['text_markdown'] # Указываем название столбца с текстом
)

In [ ]:
model.fit(X_train, y_train)

# Предсказания
predictions = model.predict(X_test)

# Оценка модели
print("Accuracy:", accuracy_score(y_test, predictions))

0:	learn: 7.7592225	total: 2m 20s	remaining: 1d 14h 55m 32s
10:	learn: 7.6917310	total: 28m 1s	remaining: 1d 17h 58m 57s


In [ ]:
def predict_tag(title, text):
    # Подготавливаем входные данные
    input_df = pd.DataFrame({'text_markdown': [text]})
    
    # Предсказываем
    predicted_tag = model.predict(input_df)
    
    return predicted_tag[0]  # Возвращает первый элемент, так как результатом будет одиночный тег

# Пример использования
predicted_tag = predict_tag('В детстве похожие камни находил на речке рядом')
print("Предсказанный тег:", predicted_tag)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

def plot_metrics(model, X_test, y_test):
    # Предсказываем вероятности
    y_scores = model.predict_proba(X_test)
    
    # Для ROC AUC нам нужны вероятности, поэтому проверяем, поддерживает ли модель predict_proba
    if hasattr(model, "predict_proba"):
        # Binarize the output labels for ROC curve
        y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
        n_classes = y_test_bin.shape[1]

        # Вычисляем ROC AUC для каждого класса
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_scores[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Рисуем ROC кривую для каждого класса
        plt.figure(figsize=(10, 8))
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %s' % (roc_auc[i], i))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

    # F1 Score
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')  # Используем weighted для многоклассовой классификации
    print("F1 Score: ", f1)


In [ ]:
def show_predicted_tags_with_probabilities(model, X_test):
    # Проверяем, поддерживает ли модель predict_proba
    if hasattr(model, "predict_proba"):
        probabilities = model.predict_proba(X_test)
        class_labels = model.classes_
        for index, prob in enumerate(probabilities):
            print(f"Sample {index + 1}:")
            for class_index, class_prob in enumerate(prob):
                print(f"   {class_labels[class_index]}: {class_prob:.4f}")
            print("\n")
    else:
        print("This model does not support probability predictions.")


In [14]:
def save_catboost_model(model, filename):
    model.save_model(filename, format='cbm')

# Пример использования:
save_catboost_model(model, 'catboost_model.cbm')


In [ ]:
from catboost import CatBoostClassifier

def load_catboost_model(filename):
    model = CatBoostClassifier()
    model.load_model(filename, format='cbm')
    return model

# Пример использования:
# loaded_catboost_model = load_catboost_model('catboost_model.cbm')
